In [ ]:
from flask import Flask, render_template, Response, request
import cv2
import numpy as np
from keras.models import load_model

app = Flask(__name__)

# Define camera setting
frameWidth = 640  
frameHeight = 480
brightness = 180
threshold = 0.80 
font = cv2.FONT_HERSHEY_SIMPLEX

# Initialize camera and model as global variables
cap = None
model = None

def initialize_camera():
    global cap
    cap = cv2.VideoCapture(0)
    cap.set(3, frameWidth)
    cap.set(4, frameHeight)
    cap.set(10, brightness)

def close_camera():
    global cap
    if cap is not None:
        cap.release()

def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def equalize(img):
    img = cv2.equalizeHist(img)
    return img

def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

def getClassName(classNo):

    if classNo == 0:
        return 'Speed Limit 20 km/h'
    elif classNo == 1:
        return 'Speed Limit 30 km/h'
    elif classNo == 2:
        return 'Speed Limit 50 km/h'
    elif classNo == 3: return 'Speed Limit 60 km/h'
    elif classNo == 4: return 'Speed Limit 70 km/h'
    elif classNo == 5: return 'Speed Limit 80 km/h'
    elif classNo == 6: return 'End of Speed Limit 80 km/h'
    elif classNo == 7: return 'Speed Limit 100 km/h'
    elif classNo == 8: return 'Speed Limit 120 km/h'
    elif classNo == 9: return 'No passing'
    elif classNo == 10: return 'No passing for vechiles over 3.5 metric tons'
    elif classNo == 11: return 'Right-of-way at the next intersection'
    elif classNo == 12: return 'Priority road'
    elif classNo == 13: return 'Yield'
    elif classNo == 14: return 'Stop'
    elif classNo == 15: return 'No vechiles'
    elif classNo == 16: return 'Vechiles over 3.5 metric tons prohibited'
    elif classNo == 17: return 'No entry'
    elif classNo == 18: return 'General caution'
    elif classNo == 19: return 'Dangerous curve to the left'
    elif classNo == 20: return 'Dangerous curve to the right'
    elif classNo == 21: return 'Double curve'
    elif classNo == 22: return 'Bumpy road'
    elif classNo == 23: return 'Slippery road'
    elif classNo == 24: return 'Road narrows on the right'
    elif classNo == 25: return 'Road work'
    elif classNo == 26: return 'Traffic signals'
    elif classNo == 27: return 'Pedestrians'
    elif classNo == 28: return 'Children crossing'
    elif classNo == 29: return 'Bicycles crossing'
    elif classNo == 30: return 'Beware of ice/snow'
    elif classNo == 31: return 'Wild animals crossing'
    elif classNo == 32: return 'End of all speed and passing limits'
    elif classNo == 33: return 'Turn right ahead'
    elif classNo == 34: return 'Turn left ahead'
    elif classNo == 35: return 'Ahead only'
    elif classNo == 36: return 'Go straight or right'
    elif classNo == 37: return 'Go straight or left'
    elif classNo == 38: return 'Keep right'
    elif classNo == 39: return 'Keep left'
    elif classNo == 40: return 'Roundabout mandatory'
    elif classNo == 41: return 'End of no passing'
    elif classNo == 42: return 'End of no passing by vechiles over 3.5 metric tons'

def detect_traffic_sign():
    while True:
        success, imgOriginal = cap.read()
        
        if not success:
            continue
        
        img = np.asarray(imgOriginal)
        img = cv2.resize(img, (32, 32))
        img = preprocessing(img)
        img = img.reshape(1, 32, 32, 1)

        predictions = model.predict(img)
        classIndex = np.argmax(predictions, axis=-1)
        probabilityValue = np.amax(predictions)

        if probabilityValue > threshold:
            class_name = getClassName(classIndex[0])
            cv2.putText(imgOriginal, str(classIndex[0]) + " " + class_name,(20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)

            cv2.putText(imgOriginal, str(round(probabilityValue * 100, 2)) + "%",(20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        _, jpeg = cv2.imencode('.jpg', imgOriginal)
        frame = jpeg.tobytes()
        yield (b'--frame\r\n'b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

@app.before_first_request
def before_first_request():
    global model
    model = load_model('my_model.h5')

@app.before_request
def before_request():
    if request.path == '/video_feed':
        initialize_camera()

@app.after_request
def after_request(response):
    if request.path != '/video_feed':
        close_camera()
    return response

@app.route('/')
def index():
    return render_template('home.html')

@app.route('/explore')
def explore():
    return render_template('home2.html')

@app.route('/video_feed')
def video_feed():
    return Response(detect_traffic_sign(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run()
    